#### Wednesday, December 6, 2023

Looking into the following HuggingFace blog article ...

[Optimum-NVIDIA - Unlock blazingly fast LLM inference in just 1 line of code](https://huggingface.co/blog/optimum-nvidia)

https://github.com/huggingface/optimum-nvidia

Dammit ... no pip package yet and "docker pull huggingface/optimum-nvidia" fails ... guess I need to wait a bit before I can give this a go ... 

In [1]:
from optimum.nvidia.pipelines import pipeline

ModuleNotFoundError: No module named 'optimum.nvidia'

[Llama 2 is here - get it on Hugging Face](https://huggingface.co/blog/llama2)

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

model = "meta-llama/Llama-2-13b-chat-hf"

model = "TheBloke/Llama-2-13b-Chat-GPTQ"

In [3]:
!ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ
models--TheBloke--Llama-2-13b-Chat-GPTQ
models--TheBloke--Python-Code-33B-GPTQ
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-hf
tmpcjh0h7gn
tmpfcrmwgx2
tmpzafytbf_
version.txt


In [ ]:
# docker cp /home/rob/Data3/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 26GB to c9b676310ea0://home/rob/Data2/huggingface/transformers

# docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--Llama-2-13b-Chat-GPTQ c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 7.26GB to c9b676310ea0://home/rob/Data2/huggingface/transformers


In [5]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--TheBloke--Llama-2-13B-chat-GPTQ
# !rm -rf /home/rob/Data2/huggingface/transformers/models--TheBloke--Llama-2-13b-Chat-GPTQ

In [7]:
!ls /home/rob/Data2/huggingface/transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


models--TheBloke--CodeLlama-34B-Instruct-GPTQ
models--TheBloke--Llama-2-13b-Chat-GPTQ
models--TheBloke--Python-Code-33B-GPTQ
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-chat-hf
models--meta-llama--Llama-2-13b-hf
tmp0m77pc0x
tmpcjh0h7gn
tmpfcrmwgx2
tmpzafytbf_
version.txt


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [4]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

# "meta-llama/Llama-2-13b-chat-hf"
# 9.4s
# 12936 MiB VRAM

# "TheBloke/Llama-2-13b-Chat-GPTQ"
# 6.9s
# 8610 MiB VRAM

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


In [5]:
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

# "TheBloke/Llama-2-13b-Chat-GPTQ"
# 6.0s

# "meta-llama/Llama-2-13b-chat-hf"
# 7m 0.6s

/transformers/src/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Keep the output from every model for comparison purposes ...

In [9]:
# "meta-llama/Llama-2-13b-chat-hf"
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

  * Yes, I can definitely recommend some shows that you might enjoy! Based on your interest in "Breaking Bad" and "Band of Brothers", I would suggest checking out the following shows:

1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This HBO series is a gritty and intense drama that explores the drug trade in Baltimore from multiple perspectives, including law enforcement, drug dealers, and politicians.
3. "Narcos" - This Netflix series tells the true story of Pablo Escobar, the infam


In [6]:
# "TheBloke/Llama-2-13b-Chat-GPTQ"
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

I understand that you have different tastes and preferences, but I can provide some recommendations based on the shows you've enjoyed. Here are a few suggestions:

1. "The Sopranos" - This critically acclaimed series is about a New Jersey mob boss, Tony Soprano, and his family. It explores themes of crime, family, and psychology.
2. "The Wire" - This gritty HBO series is set in Baltimore and explores the drug trade and its impact on the city and its residents. It features a talented ensemble cast and is known for its complex characters and storylines.
3. "Narcos" - This Netflix series tells the true story of Pablo Escobar and the Medellín cartel, and expl
